## Train model with long sentences only

In [2]:
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_problems

### Extract long sentences from data

In [3]:
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_problems
from tensor2tensor.utils import registry

import gzip # import library for reading datasets

@registry.register_problem
class OnlineRevewProjectUSYD(text_problems.Text2TextProblem):

    @property
    def approx_vocab_size(self):
        return 2**15  #15 ~32768 # 13~8k

    @property
    def is_generate_per_split(self):
        return False

    @property
    def dataset_splits(self):
    # 10% evaluation data
        return [{
            "split": problem.DatasetSplit.TRAIN,
            "shards": 9,
        }, {
            "split": problem.DatasetSplit.EVAL,
            "shards": 1,
        }]   
    
    def generate_samples(self, data_dir, tmp_dir, dataset_split):
        del data_dir
        del tmp_dir
        del dataset_split

        # load dataset
        dataset = "../../../data/reviews.json.gz"
        
        with gzip.open(dataset, 'rb') as f:
            
            for line in f:
                data = eval(line)
                # only select text contains more than or equal to 384 words
                tokens = data['review'].split()
                if len(tokens)<384: 
                    continue

                yield {
                    "inputs": data['review'],
                    "targets": data['summary']
                }
# import generate_problem # self-defined 'generate_problem.py' file in the same folder.

# define the directory for generated data
USR_DIR= './'
PROBLEM = 'OnlineRevewProjectUSYD_v6'
TMP_DIR = './tmp' 
DATA_LOC = './data' 

# Generated training data
t2t_problem = OnlineRevewProjectUSYD(PROBLEM)
t2t_problem.generate_data(DATA_LOC, TMP_DIR)

INFO:tensorflow:Generating vocab file: ./data/vocab.online_revew_project_usyd.32768.subwords
INFO:tensorflow:Trying min_count 500
INFO:tensorflow:Iteration 0
INFO:tensorflow:Processing token [environtment] took 0.13033199310302734 seconds, consider setting Text2TextProblem.max_subtoken_length to a smaller value.
INFO:tensorflow:Processing token [audiencepastorboardcongregationmomdadfamilyschoolmates] took 0.2612156867980957 seconds, consider setting Text2TextProblem.max_subtoken_length to a smaller value.
INFO:tensorflow:Processing token [rhinitisjohnsons] took 0.5244102478027344 seconds, consider setting Text2TextProblem.max_subtoken_length to a smaller value.
INFO:tensorflow:vocab_size = 103250
INFO:tensorflow:Iteration 1
INFO:tensorflow:Processing token [breakdownloadup] took 0.13013052940368652 seconds, consider setting Text2TextProblem.max_subtoken_length to a smaller value.
INFO:tensorflow:Processing token [scene19] took 0.26268649101257324 seconds, consider setting Text2TextProb

### Train with full length

In [4]:
# select the transformer model
from tensor2tensor.utils import registry
from tensor2tensor import models
from tensor2tensor.utils.trainer_lib import create_hparams
from tensor2tensor.utils.trainer_lib import create_run_config, create_experiment
import json
import timeit
# running time calculation
start = timeit.default_timer()

MODEL = 'transformer'
HPARAMS = 'transformer_prepend'
TRAIN_DIR = 'model_files_long_full'

hparams = create_hparams(HPARAMS)
hparams.batch_size = 1024 
hparams.learning_rate = 0.2 # default 0.2
hparams.num_encoder_layers = 6 # default 0
hparams.num_decoder_layers = 6 # default 0
# hparams.max_input_seq_length = 512

RUN_CONFIG = create_run_config(
    model_name=MODEL,
    model_dir=TRAIN_DIR,)

t2t_model = create_experiment(
        run_config=RUN_CONFIG,
        hparams=hparams,
        model_name=MODEL,
        problem_name=t2t_problem, #PROBLEM,
        data_dir='./data', 
        train_steps=10001, 
        eval_steps=200 
    )

# training
t2t_model.train_and_evaluate()

# running time check (running on 1GPU server)
stop = timeit.default_timer()
print('Time: {} s'.format(round(stop - start,2)))

Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
INFO:tensorflow:Configuring DataParallelism to replicate the model.
INFO:tensorflow:schedule=continuous_train_and_eval
INFO:tensorflow:worker_gpu=1
INFO:tensorflow:sync=False
INFO:tensorflow:datashard_devices: ['gpu:0']
INFO:tensorflow:caching_devices: None
INFO:tensorflow:ps_devices: ['gpu:0']
INFO:tensorflow:Using config: {'data_parallelism': <tensor2tensor.utils.expert_utils.Parallelism object at 0x7fd389f6fb38>, '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 0.95
}
allow_soft_placement: true
graph_options {
  optimizer_options {
    global_jit_level: OFF
  }
}
, '_task_type': None, '_save_summary_steps': 100, '_device_fn': None, '_protocol': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd389f6fb70>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_save_checkpoints_secs': None, '_num_ps_replica

/usr/local/lib/python3.5/dist-packages/tensorflow/python/platform/tf_logging.py:161: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  get_logger().warn(msg, *args, **kwargs)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.
INFO:tensorflow:Calling model_fn.


/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'train'
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_33049_512.bottom
Instructions for updating:
Shapes are always computed; don't use the compute_shapes as it has no effect.
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_33049_512.targets_bottom
INFO:tensorflow:Building model body
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.dense instead.
INFO:tensorflow:Transforming body output with symbol_modality_33049_512.top
INFO:tensorflow:Base learning rate: 0.200000
INFO:tensorflow:Trainable Variables Total size: 61041152
INFO:tensorflow:Non-trainable variables Total size: 5
INFO:tensorflow:Using optimizer Adam


...

INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_33049_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_33049_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_33049_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-26T03:29:00Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_files_long_full/model.ckpt-6000
INFO:tensorflow:Running local_init_op.
INFO:tenso

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_33049_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_33049_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_33049_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-26T03:46:41Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_files_long_full/model.ckpt-8000
INFO:tensorflow:Running local_init_op.
INFO:tenso

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_33049_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_33049_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_33049_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-26T04:04:17Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_files_long_full/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tens

### Train with first 256 words

In [6]:
# select the transformer model
from tensor2tensor.utils import registry
from tensor2tensor import models
from tensor2tensor.utils.trainer_lib import create_hparams
from tensor2tensor.utils.trainer_lib import create_run_config, create_experiment
import json
import timeit
# running time calculation
start = timeit.default_timer()

MODEL = 'transformer'
HPARAMS = 'transformer_prepend'
TRAIN_DIR = 'model_files_first256'

hparams = create_hparams(HPARAMS)
hparams.batch_size = 1024 
hparams.learning_rate = 0.2 # default 0.2
hparams.num_encoder_layers = 6 # default 0
hparams.num_decoder_layers = 6 # default 0
hparams.max_input_seq_length = 256

RUN_CONFIG = create_run_config(
    model_name=MODEL,
    model_dir=TRAIN_DIR,)

t2t_model_first256 = create_experiment(
        run_config=RUN_CONFIG,
        hparams=hparams,
        model_name=MODEL,
        problem_name=t2t_problem, #PROBLEM,
        data_dir='./data', 
        train_steps=10001, 
        eval_steps=200 
    )

# training
t2t_model_first256.train_and_evaluate()

# running time check (running on 1GPU server)
stop = timeit.default_timer()
print('Time: {} s'.format(round(stop - start,2)))

INFO:tensorflow:Configuring DataParallelism to replicate the model.
INFO:tensorflow:schedule=continuous_train_and_eval
INFO:tensorflow:worker_gpu=1
INFO:tensorflow:sync=False
INFO:tensorflow:datashard_devices: ['gpu:0']
INFO:tensorflow:caching_devices: None
INFO:tensorflow:ps_devices: ['gpu:0']
INFO:tensorflow:Using config: {'data_parallelism': <tensor2tensor.utils.expert_utils.Parallelism object at 0x7fd389f6ffd0>, '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 0.95
}
allow_soft_placement: true
graph_options {
  optimizer_options {
    global_jit_level: OFF
  }
}
, '_task_type': None, '_save_summary_steps': 100, '_device_fn': None, '_protocol': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd37ac88eb8>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_save_checkpoints_secs': None, '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', 't2t_device_info': {'num_async_replicas': 1}, 

/usr/local/lib/python3.5/dist-packages/tensorflow/python/platform/tf_logging.py:161: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  get_logger().warn(msg, *args, **kwargs)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'train'
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_33049_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_33049_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_33049_512.top
INFO:tensorflow:Base learning rate: 0.200000
INFO:tensorflow:Trainable Variables Total size: 61041152
INFO:tensorflow:Non-trainable variables Total size: 5
INFO:tensorflow:Using optimizer Adam


/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into model_files_first256/model.ckpt.
INFO:tensorflow:loss = 9.484326, step = 0
INFO:tensorflow:global_step/sec: 2.6598
INFO:tensorflow:loss = 7.818598, step = 100 (37.598 sec)
INFO:tensorflow:global_step/sec: 3.11306
INFO:tensorflow:loss = 7.0990424, step = 200 (32.123 sec)
INFO:tensorflow:global_step/sec: 3.11125
INFO:tensorflow:loss = 6.397411, step = 300 (32.142 sec)
INFO:tensorflow:global_step/sec: 3.11193
INFO:tensorflow:loss = 6.1933303, step = 400 (32.134 sec)
INFO:tensorflow:global_step/sec: 3.11288
INFO:tensorflow:loss = 6.048176, step = 500 (32.124 sec)
INFO:tensorflow:global_step/sec: 3.1039
INFO:tensorflow:loss = 6.1622186, step = 600 (32.218 sec)
INFO:tensorflow:global_step/sec: 3.10772
INFO:tensorflow:loss = 5.479675, step = 700 (3

...

INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_33049_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_33049_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_33049_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-26T04:56:07Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_files_first256/model.ckpt-6000
INFO:tensorflow:Running local_init_op.
INFO:tensor

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_33049_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_33049_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_33049_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-26T05:08:20Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_files_first256/model.ckpt-8000
INFO:tensorflow:Running local_init_op.
INFO:tensor

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_33049_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_33049_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_33049_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-26T05:20:33Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_files_first256/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tenso

### Train with last 256 words

In [8]:
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_problems
from tensor2tensor.utils import registry

import gzip # import library for reading datasets

@registry.register_problem
class OnlineRevewProjectUSYD_last256(text_problems.Text2TextProblem):

    @property
    def approx_vocab_size(self):
        return 2**15  #15 ~32768 # 13~8k

    @property
    def is_generate_per_split(self):
        return False

    @property
    def dataset_splits(self):
    # 10% evaluation data
        return [{
            "split": problem.DatasetSplit.TRAIN,
            "shards": 9,
        }, {
            "split": problem.DatasetSplit.EVAL,
            "shards": 1,
        }]   
    
    def generate_samples(self, data_dir, tmp_dir, dataset_split):
        del data_dir
        del tmp_dir
        del dataset_split

        # load dataset
        dataset = "../../../data/reviews.json.gz"
        
        with gzip.open(dataset, 'rb') as f:
            
            for line in f:
                data = eval(line)
                # only select text contains more than or equal to 384 words
                review = data['review']
                tokens = review.split()
                if len(tokens)<384: 
                    continue
                else: 
                    selected = tokens[-256:]
                    review = ' '.join(selected)

                yield {
                    "inputs": review,
                    "targets": data['summary']
                }
# import generate_problem # self-defined 'generate_problem.py' file in the same folder.

# define the directory for generated data
USR_DIR= './'
PROBLEM = 'OnlineRevewProjectUSYD_last256'
TMP_DIR = './tmp_last256' 
DATA_LOC = './data_last256' 

# Generated training data
t2t_problem = OnlineRevewProjectUSYD_last256(PROBLEM)
t2t_problem.generate_data(DATA_LOC, TMP_DIR)

INFO:tensorflow:Generating vocab file: ./data_last256/vocab.online_revew_project_usyd_last256.32768.subwords
INFO:tensorflow:Trying min_count 500
INFO:tensorflow:Iteration 0
INFO:tensorflow:Processing token [biblebanging] took 0.13361477851867676 seconds, consider setting Text2TextProblem.max_subtoken_length to a smaller value.
INFO:tensorflow:Processing token [zamyatins] took 0.11696743965148926 seconds, consider setting Text2TextProblem.max_subtoken_length to a smaller value.
INFO:tensorflow:Processing token [unrelatably] took 0.26570940017700195 seconds, consider setting Text2TextProblem.max_subtoken_length to a smaller value.
INFO:tensorflow:Processing token [ricaestoniagreecehungaryicelandisraeljapanlithuaniamongolianorwayromainiaspainswedenunited] took 0.5672762393951416 seconds, consider setting Text2TextProblem.max_subtoken_length to a smaller value.
INFO:tensorflow:Processing token [watziznaymegmail] took 1.1514463424682617 seconds, consider setting Text2TextProblem.max_subtok

INFO:tensorflow:Processing token [unrelatably] took 0.21663856506347656 seconds, consider setting Text2TextProblem.max_subtoken_length to a smaller value.
INFO:tensorflow:Processing token [ricaestoniagreecehungaryicelandisraeljapanlithuaniamongolianorwayromainiaspainswedenunited] took 0.5092475414276123 seconds, consider setting Text2TextProblem.max_subtoken_length to a smaller value.
INFO:tensorflow:Processing token [watziznaymegmail] took 1.0229852199554443 seconds, consider setting Text2TextProblem.max_subtoken_length to a smaller value.
INFO:tensorflow:vocab_size = 82749
INFO:tensorflow:Iteration 1
INFO:tensorflow:Processing token [weisenbergers] took 0.13029837608337402 seconds, consider setting Text2TextProblem.max_subtoken_length to a smaller value.
INFO:tensorflow:Processing token [huging] took 0.22131609916687012 seconds, consider setting Text2TextProblem.max_subtoken_length to a smaller value.
INFO:tensorflow:Processing token [unemancipated] took 0.5265529155731201 seconds, c

In [10]:
# select the transformer model
from tensor2tensor.utils import registry
from tensor2tensor import models
from tensor2tensor.utils.trainer_lib import create_hparams
from tensor2tensor.utils.trainer_lib import create_run_config, create_experiment
import json
import timeit
# running time calculation
start = timeit.default_timer()

MODEL = 'transformer'
HPARAMS = 'transformer_prepend'
TRAIN_DIR = 'model_files_last256'

hparams = create_hparams(HPARAMS)
hparams.batch_size = 1024 
hparams.learning_rate = 0.2 # default 0.2
hparams.num_encoder_layers = 6 # default 0
hparams.num_decoder_layers = 6 # default 0
hparams.max_input_seq_length = 256

RUN_CONFIG = create_run_config(
    model_name=MODEL,
    model_dir=TRAIN_DIR,)

t2t_model_last256 = create_experiment(
        run_config=RUN_CONFIG,
        hparams=hparams,
        model_name=MODEL,
        problem_name=t2t_problem, #PROBLEM,
        data_dir='./data_last256', 
        train_steps=10001, 
        eval_steps=200 
    )

# training
t2t_model_last256.train_and_evaluate()

# running time check (running on 1GPU server)
stop = timeit.default_timer()
print('Time: {} s'.format(round(stop - start,2)))

INFO:tensorflow:Configuring DataParallelism to replicate the model.
INFO:tensorflow:schedule=continuous_train_and_eval
INFO:tensorflow:worker_gpu=1
INFO:tensorflow:sync=False
INFO:tensorflow:datashard_devices: ['gpu:0']
INFO:tensorflow:caching_devices: None
INFO:tensorflow:ps_devices: ['gpu:0']
INFO:tensorflow:Using config: {'data_parallelism': <tensor2tensor.utils.expert_utils.Parallelism object at 0x7fd36e63a240>, '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 0.95
}
allow_soft_placement: true
graph_options {
  optimizer_options {
    global_jit_level: OFF
  }
}
, '_task_type': None, '_save_summary_steps': 100, '_device_fn': None, '_protocol': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd36e63a278>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_save_checkpoints_secs': None, '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', 't2t_device_info': {'num_async_replicas': 1}, 

/usr/local/lib/python3.5/dist-packages/tensorflow/python/platform/tf_logging.py:161: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  get_logger().warn(msg, *args, **kwargs)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Calling model_fn.


/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'train'
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_32700_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_32700_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_32700_512.top
INFO:tensorflow:Base learning rate: 0.200000
INFO:tensorflow:Trainable Variables Total size: 60862464
INFO:tensorflow:Non-trainable variables Total size: 5
INFO:tensorflow:Using optimizer Adam


...

INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_32700_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_32700_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_32700_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-26T07:11:25Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_files_last256/model.ckpt-6000
INFO:tensorflow:Running local_init_op.
INFO:tensorf

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_32700_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_32700_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_32700_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-26T07:23:36Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_files_last256/model.ckpt-8000
INFO:tensorflow:Running local_init_op.
INFO:tensorf

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_32700_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_32700_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_32700_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-26T07:35:49Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_files_last256/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensor

# Hyparams

In [13]:
# define the directory for generated data
USR_DIR= './'
PROBLEM = 'OnlineRevewProjectUSYD_v6'
TMP_DIR = './tmp' 
DATA_LOC = './data' 

# Generated training data
t2t_problem = OnlineRevewProjectUSYD(PROBLEM)
t2t_problem.generate_data(DATA_LOC, TMP_DIR)
# running time calculation
start = timeit.default_timer()

MODEL = 'transformer'
HPARAMS = 'transformer_prepend'
TRAIN_DIR = 'model_files_long_16head'

hparams = create_hparams(HPARAMS)
hparams.batch_size = 1024 
hparams.learning_rate = 0.2 
hparams.num_encoder_layers = 6 
hparams.num_decoder_layers = 6 
hparams.num_hidden_layers =12 # default 6
hparams.num_heads = 16 # default 8
# print(hparams)

RUN_CONFIG = create_run_config(
    model_name=MODEL,
    model_dir=TRAIN_DIR,)

t2t_model = create_experiment(
        run_config=RUN_CONFIG,
        hparams=hparams,
        model_name=MODEL,
        problem_name=t2t_problem, #PROBLEM,
        data_dir='./data', 
        train_steps=10001, 
        eval_steps=200 
    )

# training
t2t_model.train_and_evaluate()

# running time check (running on 1GPU server)
stop = timeit.default_timer()
print('Time: {} s'.format(round(stop - start,2)))

INFO:tensorflow:Found vocab file: ./data/vocab.online_revew_project_usyd.32768.subwords
INFO:tensorflow:Skipping generator because outputs files exists at ['./data/online_revew_project_usyd-unshuffled-train-00000-of-00009', './data/online_revew_project_usyd-unshuffled-train-00001-of-00009', './data/online_revew_project_usyd-unshuffled-train-00002-of-00009', './data/online_revew_project_usyd-unshuffled-train-00003-of-00009', './data/online_revew_project_usyd-unshuffled-train-00004-of-00009', './data/online_revew_project_usyd-unshuffled-train-00005-of-00009', './data/online_revew_project_usyd-unshuffled-train-00006-of-00009', './data/online_revew_project_usyd-unshuffled-train-00007-of-00009', './data/online_revew_project_usyd-unshuffled-train-00008-of-00009', './data/online_revew_project_usyd-unshuffled-dev-00000-of-00001']
INFO:tensorflow:Skipping shuffle because output files exist
INFO:tensorflow:Configuring DataParallelism to replicate the model.
INFO:tensorflow:schedule=continuous_tr

/usr/local/lib/python3.5/dist-packages/tensorflow/python/platform/tf_logging.py:161: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  get_logger().warn(msg, *args, **kwargs)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Setting T2TModel mode to 'train'
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_33049_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_33049_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_33049_512.top
INFO:tensorflow:Base learning rate: 0.200000
INFO:tensorflow:Trainable Variables Total size: 61041152
INFO:tensorflow:Non-trainable variables Total size: 5
INFO:tensorflow:Using optimizer Adam


/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into model_files_long_16head/model.ckpt.
INFO:tensorflow:loss = 9.585581, step = 0
INFO:tensorflow:global_step/sec: 1.96846
INFO:tensorflow:loss = 8.055903, step = 100 (50.799 sec)
INFO:tensorflow:global_step/sec: 2.2542
INFO:tensorflow:loss = 7.382488, step = 200 (44.363 sec)
INFO:tensorflow:global_step/sec: 2.40169
INFO:tensorflow:loss = 6.34474, step = 300 (41.636 sec)
INFO:tensorflow:global_step/sec: 2.24597
INFO:tensorflow:loss = 6.8033648, step = 400 (44.524 sec)
INFO:tensorflow:global_step/sec: 2.24042
INFO:tensorflow:loss = 6.0605564, step = 500 (44.636 sec)
INFO:tensorflow:global_step/sec: 2.26859
INFO:tensorflow:loss = 5.78826, step = 600 (44.079 sec)
INFO:tensorflow:global_step/sec: 2.20444
INFO:tensorflow:loss = 5.8641477, step = 700 

...

INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_33049_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_33049_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_33049_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-26T08:57:36Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_files_long_16head/model.ckpt-6000
INFO:tensorflow:Running local_init_op.
INFO:ten

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_33049_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_33049_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_33049_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-26T09:17:14Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_files_long_16head/model.ckpt-8000
INFO:tensorflow:Running local_init_op.
INFO:ten

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_33049_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_33049_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_33049_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-26T09:37:26Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_files_long_16head/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:te